## S&P Global - AUTOMATIC BASELINE CORRECTION
### Misael M. Morales, Carlos Torres-Verdin, and Michael Pyrcz, UT-Austin; Murray Christie, Vladimir Rabinovich, S&P Global
#### 2024, Digital Reservoir Characterization Technology (DiReCT)
***

In [ ]:
from auto_baseline_correction import *

***
# LOG ANALYSIS
***

In [ ]:
# instantiate the LogAnalysis class
spl = SPLogAnalysis()
spl.__dict__

In [ ]:
# ### WARNING: this takes a long time to run, only do it if you need to
# headers = spl.read_all_headers()

# Plot the spatial distribution of wells and the selected property
spl.plot_ccs_sand_wells(figsize=(8,3), value='POROSITY', cmap='jet')

# Plot a well deviation survey for a given well
spl.plot_survey(figsize=(10,3), fname='427064023000_DIRSUR_NAD27(USFEET)US-SPC27-EXACT(TX-27SC).TXT')

# Plot the full well log with multiple tracks
# also plots the autocorrelation plot and ARIMA model curve
spl.plot_well(figsize=(10,8), well_name='17700004060000', curve='SP', order=(5,1,0))

***
# SP BASELINE CORRECTION
***

In [ ]:
# instanstiate the BaselineCorrection class
blc = BaselineCorrection()
blc.__dict__

In [ ]:
# Load the .LAS files from scratch (preload=False). 
# This means you don't have a pre-existing .npy file with the pre-loaded data.
# The .LAS files are in your 'data' folder, and this will output a log_data.npy file with the preprocessed logs.
blc.load_logs(preload      = True,                  #change me to False if you don't have a pre-existing .npy file 
              preload_file = 'Data/log_data.npy',   #change me (if preload_file is not 'Data/log_data.npy')
              folder       = 'Data/UT Export 9-19', #change me (if preload folder is not 'Data/UT Export 9-19' and preload=False)
              save_file    = 'Data/log_data.npy',   #change me to save your preprocessed raw logs [.LAS -> .npy]
              showfig      = True,
              )

# Process the numpy logs. 
# This will scale the data according to the chosen scaler, and perform random train_test_split
# scaler can be either ('standard', 'minmax', or 'none')
blc.scale_and_random_split(scaler='standard', 
                           test_size=0.227, 
                           showfig=True,
                          )

# Make the baseline correction NN model.
# If pretrained:
#    - the model will be loaded from the file pretrained='baseline_correction_model.keras'
# Else:
#    - we construct a model from scratch based on the auxiliary functions in the class
#    - and train the model on the training data
blc.make_model(pretrained   = 'baseline_correction_model.keras', #change me to None to train from scratch
               show_summary = False, 
               kernel_size  = 15, 
               dropout      = 0.2,
               depths       = [16,32,64], 
               optimizer    = 'adam',
               lr           = 1e-3,
               loss         = 'mse',
               metrics      = ['mse'],
               epochs       = 100,
               batch_size   = 30,
               valid_split  = 0.25,
               verbose      = True,
               figsize      = (10,5),
               )

# Make predictions on the test data and visualize results
blc.make_predictions(showfig=True, xlim=(-5,5))

***
# Transfer Learning Baseline Correction
***

In [ ]:
# instantiate the TransferLearning-BaselineCorrection class
tlc = TransferLearning()
tlc.__dict__

In [ ]:
# Run SP* and Csh estimation
# Using the pre-trained model from the BaselineCorrection class, we can predict the baseline-correct SP log
# and the Csh for every well log in the data folder. The .LAS files are updated with the 2 new curves, and 
# saved in a 'postprocess' folder.
tlc.make_transfer_prediction()

In [ ]:
# Plot results of the SP baseline correction and Csh estimation for a given well
tlc.plot_transfer_results(filenum = 309,
                          figsize = (10,8),
                          showfig = True,
                        )

***
# END